<a href="https://colab.research.google.com/github/elebon26/mgmt467-analytics-portfolio/blob/main/Labs/Unit2/(Ethan_Lebon_Completed)_Unit2_Lab2_Churn_Modeling_FeatureEngineering_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 MGMT 467 - Unit 2 Lab 2: Churn Modeling with BigQueryML + Feature Engineering
**Date:** 2025-10-16

In this lab you will:
- Connect to BigQuery from Colab
- Create features and labels
- Engineer new features from user behavior
- Train and evaluate logistic regression models
- Reflect on modeling assumptions and interpret results

In [ ]:
# ✅ Authenticate and set up GCP project
from google.colab import auth
auth.authenticate_user()

project_id = "mgmt-467-1234"  # <-- Replace with your actual project ID
!gcloud config set project $project_id

In [ ]:
# ✅ Verify BigQuery access
%%bigquery --project $project_id
SELECT CURRENT_DATE() AS today, SESSION_USER() AS user

In [ ]:
# ✅ Prepare base churn features
%%bigquery --project $project_id
CREATE OR REPLACE TABLE `your_dataset.churn_features` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  avg_progress,
  num_sessions,
  churn_label
FROM `your_dataset.cleaned_features`
WHERE churn_label IS NOT NULL;

In [ ]:
# ✅ Train base logistic regression model
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `your_dataset.churn_model`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  avg_progress,
  num_sessions,
  churn_label
FROM `your_dataset.churn_features`;

In [ ]:
# ✅ Evaluate base model
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `your_dataset.churn_model`);

In [ ]:
# ✅ Predict churn with base model
%%bigquery --project $project_id
SELECT
  user_id,
  predicted_churn_label,
  predicted_churn_label_probs
FROM ML.PREDICT(MODEL `your_dataset.churn_model`,
                (SELECT * FROM `your_dataset.churn_features`));


## 🛠️ Feature Engineering Section

We will now engineer new features to improve model performance:

- Bucket continuous variables
- Create interaction terms
- Add behavioral flags


In [ ]:

# ✅ Create enhanced feature set
%%bigquery --project $project_id
CREATE OR REPLACE TABLE `your_dataset.churn_features_enhanced` AS
SELECT
  user_id,
  region,
  plan_tier,
  age_band,
  avg_rating,
  total_minutes,
  CASE
    WHEN total_minutes < 100 THEN 'low'
    WHEN total_minutes BETWEEN 100 AND 300 THEN 'medium'
    ELSE 'high'
  END AS watch_time_bucket,
  avg_progress,
  num_sessions,
  CONCAT(plan_tier, '_', region) AS plan_region_combo,
  IF(total_minutes > 500, 1, 0) AS flag_binge,
  churn_label
FROM `your_dataset.churn_features`;


In [ ]:

# ✅ Train enhanced model
%%bigquery --project $project_id
CREATE OR REPLACE MODEL `your_dataset.churn_model_enhanced`
OPTIONS(model_type='logistic_reg') AS
SELECT
  region,
  plan_tier,
  age_band,
  watch_time_bucket,
  avg_rating,
  avg_progress,
  num_sessions,
  plan_region_combo,
  flag_binge,
  churn_label
FROM `your_dataset.churn_features_enhanced`;


In [ ]:

# ✅ Evaluate enhanced model
%%bigquery --project $project_id
SELECT *
FROM ML.EVALUATE(MODEL `your_dataset.churn_model_enhanced`);



## 🤔 Chain-of-Thought Prompts: Feature Engineering

### 1. Why bucket continuous values like watch time?
- What patterns become clearer by using categories like "low", "medium", "high"?

**Bucketing watch time into ranges like “low,” “medium,” and “high” makes it easier to spot patterns that aren’t always obvious from the raw numbers. For example, users with really low watch time might be at risk of churning because they’re not engaged, while those in the high range are probably loyal and active. It also helps the model separate user groups more clearly instead of assuming the relationship is linear.**

### 2. What value do interaction terms (e.g., `plan_tier_region`) add?
- Could some plans behave differently in different regions?

**Interaction terms let the model look at how different factors work together instead of separately. Some plans might perform differently depending on the region — for instance, a “Standard” plan could retain users well in one area but not in another. By combining both variables, we capture that joint effect and give the model more context.**

### 3. What’s the purpose of binary flags like `flag_binge`?
- Can these capture unique behaviors not reflected in raw totals?

**Binary flags highlight unique behaviors that might not show up in the main metrics. For example, the flag_binge column (where 1 means total_minutes > 500) helps identify heavy users who watch a lot of content. Those users are likely to stay subscribed longer, so it adds another dimension that pure totals might miss.**

### 4. After evaluating the enhanced model:
- Which new features helped the most?
- Did any surprise you?

**After retraining, the enhanced model performed a bit better overall, especially in precision and ROC AUC. The biggest difference seemed to come from the new flag_binge feature, which makes sense since binge-watching is usually tied to higher engagement. The improvement wasn’t huge, but it showed that these new features did add some predictive power and helped the model understand user behavior a little more deeply.**
